In [109]:
import pandas as pd

data = pd.read_parquet("data/test-00000-of-00001.parquet")
data.head(15)

,qid,video_id,question_type,capability,question,duration,question_prompt,answer,youtube_url
0,0008-0,sj81PWrerDk,Primary Open-ended Question,Plot Attribute (Montage),What is the difference between the action of t...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
1,0008-1,sj81PWrerDk,Paraphrased Open-ended Question,Plot Attribute (Montage),Can you describe how the actions of the last p...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
2,0008-2,sj81PWrerDk,Correctly-led Open-ended Question,Plot Attribute (Montage),Did the last person open the bottle without us...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
3,0008-3,sj81PWrerDk,Wrongly-led Open-ended Question,Plot Attribute (Montage),Did the last person in the video open the bott...,8.85,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/sj81PWrerDk
4,0008-7,sj81PWrerDk,Multiple-choice Question with a Single Correct...,Plot Attribute (Montage),How does the last person in the video open the...,8.85,E. None of the above\nSelect one best answer t...,,https://www.youtube.com/shorts/sj81PWrerDk
5,0012-0,AGCyLqLuUJ0,Primary Open-ended Question,Element Counting,How many robot figures appear in the video?,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0
6,0012-1,AGCyLqLuUJ0,Paraphrased Open-ended Question,Element Counting,What is the total number of robot figures show...,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0
7,0012-2,AGCyLqLuUJ0,Correctly-led Open-ended Question,Element Counting,Does the video display four robot figures in t...,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0
8,0012-3,AGCyLqLuUJ0,Wrongly-led Open-ended Question,Element Counting,Are there only two robot figures shown in the ...,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0
9,0012-7,AGCyLqLuUJ0,Multiple-choice Question with a Single Correct...,Element Counting,How many robot figures are shown in the video?...,20.67,E. None of the above\nSelect one best answer t...,,https://www.youtube.com/shorts/AGCyLqLuUJ0


In [86]:
qid = data['qid']
video_id = data['video_id']
question_type = data['question_type']
capability = data['capability']
question = data['question']
duration = data['duration']
question_prompt = data['question_prompt']
answer = data['answer']
youtube_url = data['youtube_url']

In [ ]:
# data.groupby(data.qid.str[:4]).sum()

In [88]:
question_type.value_counts()

question_type
Primary Open-ended Question                              300
Paraphrased Open-ended Question                          300
Correctly-led Open-ended Question                        300
Wrongly-led Open-ended Question                          300
Multiple-choice Question with a Single Correct Answer    300
Name: count, dtype: int64

In [89]:
capability.value_counts()

capability
Element Counting                                           155
Objective Causality (Videography Phenomenon & Illusion)    130
Positional Relationship                                    120
Event Localization                                         110
Character Motivation Causality                             105
Element Attributes (Optical Illusion)                      100
Event Counting                                             100
Element Localization                                        90
Plot Attribute                                              90
Displacement Attribute                                      75
Professional Knowledge                                      70
Character Reaction Causality                                65
Character Emotion Attribute                                 60
Objective Causality                                         55
Element Attributes                                          55
Local Event Attribute                       

# Check Youtube URL for Valid Videos (Some may be terminated, private)

In [110]:
import requests

def check_youtube_video(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        session = requests.Session()
        response = session.get(url, headers=headers, timeout=5)  # Timeout added

        if response.status_code == 200:
            page_content = response.text.lower()

            # Check for error messages in the page content
            errors = [
                "video unavailable",
                "this video isn't available",
                "account has been terminated",
                "video has been removed",
                "video is private",
            ]
            
            return "Valid" if not any(error in page_content for error in errors) else "Invalid"
    
    except requests.RequestException as e:
        print(f"Error checking URL {url}: {e}")
    
    return "Invalid"
print(check_youtube_video("https://www.youtube.com/shorts/sj81PWrerDk"))
print(check_youtube_video("https://www.youtube.com/shorts/AGCyLqLuUJ0"))

Valid
Invalid


In [93]:
# for index, i in enumerate(data['youtube_url'].unique()):
#   response = requests.get(i)
#   if response.status_code == 200 and check_youtube_video(str(i)) == True:
#     print(f"Video {index} with link {i} is valid ")
#     data.loc[index:index+4, "Valid"] = "Valid"
#   else:
#     print(f"Video {index} with link {i} is invalid ")
#     data.loc[index:index+4, "Valid"] = "Invalid"

In [112]:
# Initialize 'Valid' column
data['Valid'] = ""

validity_dict = {}

# Check each unique YouTube URL and store the result
for url in data['youtube_url'].unique():
    print(f"Checking URL: {url}")
    validity_dict[url] = check_youtube_video(url)

# Apply results to DataFrame
data['Valid'] = data['youtube_url'].map(validity_dict)

Checking URL: https://www.youtube.com/shorts/sj81PWrerDk
Checking URL: https://www.youtube.com/shorts/AGCyLqLuUJ0
Checking URL: https://www.youtube.com/shorts/4mKxze5P9gQ
Checking URL: https://www.youtube.com/shorts/1SiTuYb506o
Checking URL: https://www.youtube.com/shorts/KyyDh9NP_lc
Checking URL: https://www.youtube.com/shorts/sE-KTqE9Dek
Checking URL: https://www.youtube.com/shorts/MSkL7A2yyOY
Checking URL: https://www.youtube.com/shorts/_MXxJT8Mk4k
Checking URL: https://www.youtube.com/shorts/crJZF1-5ryE
Checking URL: https://www.youtube.com/shorts/Z-x3kXiyOFc
Checking URL: https://www.youtube.com/shorts/aUDYWYqtAR4
Checking URL: https://www.youtube.com/shorts/FdkXmy42Qv8
Checking URL: https://www.youtube.com/shorts/2uuLeoxlZOo
Checking URL: https://www.youtube.com/shorts/uupXcgeP0-w
Checking URL: https://www.youtube.com/shorts/XlROzogZpVM
Checking URL: https://www.youtube.com/shorts/NmqIqJsdIuk
Checking URL: https://www.youtube.com/shorts/7t4cNOpTuCY
Checking URL: https://youtube.c

In [118]:
data['Valid'].value_counts()

Valid
Valid      1445
Invalid      55
Name: count, dtype: int64

In [128]:
data[data['Valid'] == 'Invalid']

,qid,video_id,question_type,capability,question,duration,question_prompt,answer,youtube_url,Valid
5,0012-0,AGCyLqLuUJ0,Primary Open-ended Question,Element Counting,How many robot figures appear in the video?,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0,Invalid
6,0012-1,AGCyLqLuUJ0,Paraphrased Open-ended Question,Element Counting,What is the total number of robot figures show...,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0,Invalid
7,0012-2,AGCyLqLuUJ0,Correctly-led Open-ended Question,Element Counting,Does the video display four robot figures in t...,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0,Invalid
8,0012-3,AGCyLqLuUJ0,Wrongly-led Open-ended Question,Element Counting,Are there only two robot figures shown in the ...,20.67,Please state your answer with a brief explanat...,,https://www.youtube.com/shorts/AGCyLqLuUJ0,Invalid
9,0012-7,AGCyLqLuUJ0,Multiple-choice Question with a Single Correct...,Element Counting,How many robot figures are shown in the video?...,20.67,E. None of the above\nSelect one best answer t...,,https://www.youtube.com/shorts/AGCyLqLuUJ0,Invalid
90,0133-0,u2HvKEGq3Ik,Primary Open-ended Question,Event Localization,What does the final scene depicts?,60.05,Please state your answer with a brief explanat...,,https://youtube.com/shorts/u2HvKEGq3Ik,Invalid
91,0133-1,u2HvKEGq3Ik,Paraphrased Open-ended Question,Event Localization,What is shown in the final scene of the video?,60.05,Please state your answer with a brief explanat...,,https://youtube.com/shorts/u2HvKEGq3Ik,Invalid
92,0133-2,u2HvKEGq3Ik,Correctly-led Open-ended Question,Event Localization,Does the final scene depict three men performi...,60.05,Please state your answer with a brief explanat...,,https://youtube.com/shorts/u2HvKEGq3Ik,Invalid
93,0133-3,u2HvKEGq3Ik,Wrongly-led Open-ended Question,Event Localization,Is the final scene a demonstration of a spinni...,60.05,Please state your answer with a brief explanat...,,https://youtube.com/shorts/u2HvKEGq3Ik,Invalid
94,0133-7,u2HvKEGq3Ik,Multiple-choice Question with a Single Correct...,Event Localization,What does the final scene depict?\nA. A dance ...,60.05,E. None of the above\nSelect one best answer t...,,https://youtube.com/shorts/u2HvKEGq3Ik,Invalid
